In [35]:
# Importing All the Important Libraries 
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [36]:
## Importing Dates From Helper Folder Whose 'DayofWeek' and 'week' Columns Designed in Such a way, We can Use Those Element for Navigating Through Calender of Website Using XPATH
Dates=pd.read_excel(r"./helper/Dates.xlsx")
Dates

,Date,Month,year,Day,Day_in_month,DayofWeek,week
0,2023-03-31,Mar,2023,Friday,31,6,5
1,2023-03-30,Mar,2023,Thursday,30,5,5
2,2023-03-29,Mar,2023,Wednesday,29,4,5
3,2023-03-28,Mar,2023,Tuesday,28,3,5
4,2023-03-27,Mar,2023,Monday,27,2,5
...,...,...,...,...,...,...,...
419,2022-02-05,Feb,2022,Saturday,5,7,1
420,2022-02-04,Feb,2022,Friday,4,6,1
421,2022-02-03,Feb,2022,Thursday,3,5,1
422,2022-02-02,Feb,2022,Wednesday,2,4,1


In [37]:
## Making Some Changes Because of Ambiguity in Calendar of Website
Dates['week']=np.where(Dates['Month'].isin(["Jan","May"]),Dates['week']+1,Dates['week'])

In [38]:
# Checking The Data
Dates = Dates.head(10)
Dates

,Date,Month,year,Day,Day_in_month,DayofWeek,week
0,2023-03-31,Mar,2023,Friday,31,6,5
1,2023-03-30,Mar,2023,Thursday,30,5,5
2,2023-03-29,Mar,2023,Wednesday,29,4,5
3,2023-03-28,Mar,2023,Tuesday,28,3,5
4,2023-03-27,Mar,2023,Monday,27,2,5
5,2023-03-26,Mar,2023,Sunday,26,1,5
6,2023-03-25,Mar,2023,Saturday,25,7,4
7,2023-03-24,Mar,2023,Friday,24,6,4
8,2023-03-23,Mar,2023,Thursday,23,5,4
9,2023-03-22,Mar,2023,Wednesday,22,4,4


In [39]:
# Initializing an instance of Website, from which we have to scrape data
website='https://vegetablemarketprice.com/market/noida/today'

In [47]:
Data_list=[]  # List for storing Scraped element, Each row of Data as one Dictionary 

driver = webdriver.Chrome(options=webdriver.ChromeOptions() ) # Opening Browser
sleep(1)

driver.maximize_window() # Maximizing the windows
driver.get(website)      # Loading the Website
sleep(1)



for index in Dates.index:  # iterating through all the dates 

    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[1]").click()  #Calendar dropdown
    sleep(1)


    # Making an Instance which is Combination of Month and Year to help us stop changing months 
    Desired_month= str(Dates['Month'][index]+ ' '+ str(Dates['year'][index]))


    # click until we reach to desired month's calender
    while driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[2]").text != Desired_month :
        driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[1]").click()
        sleep(0.1)

    week_row=Dates['week'][index]  #Making Instance Which we will use in Xpath To Select Week Row
    Day_cell=Dates['DayofWeek'][index] # Making Instance Which we will use in Xpath To Select Day of Week
    driver.find_element(By.XPATH,f'/html/body/div[4]/div[2]/div[1]/table/tbody/tr[{week_row}]/td[{Day_cell}]').click() # Click on Selected Date



    # wait untill Price table Loads for selected date
    try:
        myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,"todayVegetableTableRows")))
    except TimeoutException:
        print('Page is taking Too Much Time')


    # Stroring Each Row of Table as Element by Finding using ClassName
    elements =driver.find_elements(By.CLASS_NAME,"todayVegetableTableRows")
    sleep(1)


    ## Iterating through each element(row) of the Table 
    for element in elements:
        id=element.get_attribute('id') # initializing an Id Instance For Subsequent XPATH Codes

        
        # All the instances Which We want to Scrape
        Vegetable=driver.find_element(By.XPATH,'//*[@id="{}"]/td[2]'.format(str(id)))
        Wholesale=driver.find_element(By.XPATH,'//*[@id="{}"]/td[3]'.format(str(id)))
        Retail=driver.find_element(By.XPATH,'//*[@id="{}"]/td[4]'.format(str(id)))
        Shopping=driver.find_element(By.XPATH,'//*[@id="{}"]/td[5]'.format(str(id)))
        Units=driver.find_element(By.XPATH,'//*[@id="{}"]/td[6]'.format(str(id)))
        Date= Dates['Date'][index]
        

        # Storing All those Scraped Elements in List
        Data_list.append({'Date': Date,
            'Vegetable':Vegetable.text,
              'Wholesale Price':Wholesale.text,
              'Retail Price':Retail.text,
              'Shopping Mall Price':Shopping.text,
              'Units':Units.text})
        sleep(0.01)

    print(index)

print("Scrapping Done")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [68]:
## Converting List into DataFrame
df=pd.DataFrame(Data_list)

In [64]:
## Viewing Scraped Data in Proper Tabular Form
df.head(10)

,Date,Vegetable,Wholesale Price,Retail Price,Shopping Mall Price,Units
0,2023-03-31,Onion Big,₹18,₹21 - 23,₹22 - 30,1kg
1,2023-03-31,Onion Small,₹55,₹63 - 70,₹66 - 91,1kg
2,2023-03-31,Tomato,₹21,₹24 - 27,₹25 - 35,1kg
3,2023-03-31,Green Chilli,₹41,₹47 - 52,₹49 - 68,1kg
4,2023-03-31,Beetroot,₹28,₹32 - 36,₹34 - 46,1kg
5,2023-03-31,Potato,₹32,₹37 - 41,₹38 - 53,1kg
6,2023-03-31,Raw Banana (Plantain),₹10,₹12 - 13,₹12 - 17,1kg
7,2023-03-31,Amaranth Leaves,₹14,₹16 - 18,₹17 - 23,1kg
8,2023-03-31,Amla,₹50,₹58 - 64,₹60 - 83,1kg
9,2023-03-31,Ash gourd,₹20,₹23 - 25,₹24 - 33,1kg


In [59]:
# Checking Shape of Scraped Data
df.shape

(22896, 6)

In [61]:
# Checking for Duplicates 
df.duplicated().sum()

0

In [69]:
df["Wholesale Price"]=df["Wholesale Price"].str.replace('₹','').astype(np.int64)  ##Removing Rupee Sign from Wholesale Price And Converting it in int DataType
df["Retail Price"]=df["Retail Price"].str.replace('₹','')                         ##Removing Rupee Sign from Wholesale Price
df["Shopping Mall Price"]=df["Shopping Mall Price"].str.replace('₹','')           ##Removing Rupee Sign from Wholesale Price


# Since Retail Price was in Range taking Mean of Retails of Lower and Higher Value
df["Retail Price"]=(df["Retail Price"].str.split('-',expand=True)[1].astype(np.int64) +df["Retail Price"].str.split('-',expand=True)[0].astype(np.int64) )//2


# Since Shopping Mall Price was in Range taking Mean of Retails of Lower and Higher Value
df["Shopping Mall Price"]=(df["Shopping Mall Price"].str.split('-',expand=True)[1].astype(np.int64) +df["Shopping Mall Price"].str.split('-',expand=True)[0].astype(np.int64) )//2

In [71]:
# Checking Quality
df.head(10)

,Date,Vegetable,Wholesale Price,Retail Price,Shopping Mall Price,Units
0,2023-03-31,Onion Big,18,22,26,1kg
1,2023-03-31,Onion Small,55,66,78,1kg
2,2023-03-31,Tomato,21,25,30,1kg
3,2023-03-31,Green Chilli,41,49,58,1kg
4,2023-03-31,Beetroot,28,34,40,1kg
5,2023-03-31,Potato,32,39,45,1kg
6,2023-03-31,Raw Banana (Plantain),10,12,14,1kg
7,2023-03-31,Amaranth Leaves,14,17,20,1kg
8,2023-03-31,Amla,50,61,71,1kg
9,2023-03-31,Ash gourd,20,24,28,1kg


In [ ]:
# Saving the CSV file in Scraped Data Folder
df.to_csv('./Scraped Data/CSV_Noida.csv')